# Extract CCD VisitTables  in  OGA


and Save Extracted Visits into a pandas dataframe file

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/01/01
- update : 2024/02/24


In [ ]:
! eups list -s | grep LOCAL

In [ ]:
! eups list -s lsst_distrib

In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [ ]:
#filevisit_out="ccdVisitTable_202312.csv"
filevisit_out="ccdVisitTable_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [ ]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def get_bbox_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    ramin,ramax,decmin,decmax in decimal degrees
    """

    xmin = bbox.beginX
    xmax = bbox.endX
    ymin = bbox.beginY
    ymax = bbox.endY
    
    radec_ll = wcs.pixelToSky(xmin, ymin)
    radec_ur = wcs.pixelToSky(xmax, ymax)
        
    return radec_ur.getRa().asDegrees(),radec_ll.getRa().asDegrees(), radec_ll.getDec().asDegrees(),radec_ur.getDec().asDegrees() 

In [ ]:
def isradec_inbbox_radec(ra,dec,wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    ra: ra in degree
    dec: dec in degree
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    Bool
    
    """

    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(wcs, bbox)

    assert RAMIN < RAMAX
    assert DECMIN < DECMAX

    flag = (ra >= RAMIN) and (ra<=RAMAX) and (dec>= DECMIN) and (dec <= DECMAX) 
    return flag

In [ ]:
def FindTractAndPatch(row):
    """
    Apply this function on ccdvisitTable dataframe to find the tract and patch for each visit
    """
    tractsId_list = list(tracts_patchs_dict.keys())   
    try:
        ra = row["ra"]
        dec = row["dec"]
        selectFlag = False
        # loop on tract
        for tractID in tractsId_list:
            wcs = tractsInfoWCS_dict[tractID]
            tractInfo = tractsInfo_dict[tractID] 
            patches_selected = list(tracts_patchs_dict[tractID])
            # loop on patches
            for patch in tractInfo:
                patchID = patch.getSequentialIndex()
                if patchID in patches_selected:
                    ibb=patch.getInnerBBox()
                    flag = isradec_inbbox_radec(ra,dec,wcs, ibb)
                    selectFlag =  selectFlag or flag
                    if selectFlag:
                        return pd.Series([tractID,patchID])
        return pd.Series([0,0])           
                
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)          # __str__ allows args to be printed directly,
        return pd.Series([0,0])  
        

## Give the collection

In [ ]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=my_collection)

## Config

In [ ]:
#Auxtel2023_tracts = [ 5616,5634,5615,10644,383,8188,3863,455,452,5839,10643,382]

In [ ]:
Auxtel2023_tracts = [382,383,452,455,
                     3863,3864,
                     5614,5615,5616,5634,5839,
                     #8188,
                     10643,10644]

In [ ]:
with open('../data/202402/Latiss_tractspatchs_202402.pickle', 'rb') as handle:
    tracts_patchs_dict = pickle.load(handle)

In [ ]:
tracts_patchs_dict

In [ ]:
#tract_selected = 5615
#patches_selected = [294,313]
#suptitle = f"CCD visit in tract {tract_selected} and patches {patches_selected[0]} and {patches_selected[1]}"

### Find tract

In [ ]:
#for tractInfo in skymap:
#    tractID = tractInfo.getId()
#    if tractID == tract_selected:
#        break

In [ ]:
#tractInfo

In [ ]:
# WCS for this patch
#tWCS=tractInfo.getWcs()

### Find Tract

In [ ]:
tractsInfo_dict = {}
tractsInfoWCS_dict = {}
tractsId_list = list(tracts_patchs_dict.keys())

# loop to select the good tracts
for tractInfo in skymap:
    tractID = tractInfo.getId()
    # is this tract in Auxtel tract list
    if tractID in tractsId_list:
        # keep tractInfo
        tractsInfo_dict[tractID] = tractInfo 
        # keep WCS
        tractsInfoWCS_dict[tractID] = tractInfo.getWcs()

### Find patches

In [ ]:
#all_patches = []
#for patch in tractInfo:
#    patchID = patch.getSequentialIndex()
#    if patchID in patches_selected:
#        all_patches.append(patch)        

In [ ]:
#all_patches

In [ ]:
#for patch in all_patches:
#    patchID = patch.getSequentialIndex()
#    ibb=patch.getInnerBBox()
#    print("patch ",patchID,"ibb=",ibb)
#    RAMIN,RAMAX,DECMIN,DECMAX = get_bbox_radec(tWCS, ibb)

# Sources

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='ccdVisitTable', collections=my_collection, where= "instrument='LATISS'")

In [ ]:
for i, ref in enumerate(datasetRefs):
   
    print("========================datasetType = ccdVisitTable ============================================")
    print("fullId..................:",ref.dataId.full)
    

In [ ]:
visit_table = butler.get('visitTable', dataId={'instrument': 'LATISS', 'detector': 0}, collections = my_collection )    
visit_table

In [ ]:
ccd_visit_table = butler.get('ccdVisitTable', dataId={'instrument': 'LATISS', 'detector': 0}, collections = my_collection )    
ccd_visit_table

### Compute night obs

In [ ]:
ccd_visit_table["nightObs"] =ccd_visit_table.apply(lambda x: x['visitId']//100_000, axis=1)

### Compute datetime to plot vs time

In [ ]:
ccd_visit_table["Time"] = pd.to_datetime(ccd_visit_table['obsStart'])

#### Compute airmass

In [ ]:
ccd_visit_table["airmass"] = ccd_visit_table["zenithDistance"].apply(lambda x: 1/np.cos(np.pi/180.*x))

### Find those visits in the selected tract and patches

In [ ]:
ccd_visit_table[["tractID","patchID"]] = ccd_visit_table.apply( FindTractAndPatch, axis=1,result_type ='expand')

In [ ]:
ccd_visit_table.head()

In [ ]:
ccd_visit_table.tail()

## Find those visits in the selected tract and patches

### Patch 294 and 313

In [ ]:
tract_selected = 5615
patch_selected = 294
#patch_selected = 313
suptitle = f"CCD visit in tract {tract_selected} and patch {patch_selected}"

In [ ]:
ccd_visit_table1 = ccd_visit_table
ccd_visit_table1["selectedFlag"] = ccd_visit_table1[["tractID","patchID"]].apply(lambda x: x[0] == tract_selected and x[1] == patch_selected,axis =1, raw= True )

In [ ]:
ccd_visit_table_selected1 = ccd_visit_table1[ccd_visit_table1["selectedFlag"]]

In [ ]:
ccd_visit_table_selected1.drop(["selectedFlag"],axis=1,inplace=True) 

In [ ]:
ccd_visit_table_selected1.columns

In [ ]:
df = ccd_visit_table_selected1

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,figsize=(16,10))

df[df["band"]=="g"].plot(x="Time",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax1,grid=True,legend=False)
ax1.set_title("zeroPoint")


df[df["band"]=="g"].plot(x="Time",y="zenithDistance",marker='o',c="g",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="zenithDistance",marker='o',c="r",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="zenithDistance",marker='o',c="y",lw=0.0,ax=ax2,grid=True,legend=False)
ax2.set_title("zenithDistance")

df[df["band"]=="g"].plot(x="Time",y="airmass",marker='o',c="g",lw=0.0,ax=ax3,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="airmass",marker='o',c="r",lw=0.0,ax=ax3,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="airmass",marker='o',c="y",lw=0.0,ax=ax3,grid=True,legend=False)
ax3.set_title("airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()

## Plot number of visit per day

### Plot number of visit per day for patch 294

In [ ]:
ccd_visit_table1["dayobsMJD"]=ccd_visit_table1["obsStartMJD"].apply(np.int64)

In [ ]:
ccd_visit_table1.sort_values(by=['nightObs',"obsStartMJD"])

In [ ]:
#ccd_visit_table1.sort_values(by=['dayobsMJD']).count()
ccd_visit_table_counts = ccd_visit_table1.groupby(['nightObs','band']).count()["visitId"]

In [ ]:
type(ccd_visit_table_counts)

In [ ]:
ccd_visit_table_counts.name='counts'
ccd_visit_table_counts

In [ ]:
import matplotlib.ticker
fig,ax = plt.subplots(1,1,figsize=(16,10))
ccd_visit_table.plot.hist(column=["nightObs"], by="band", bins=200,grid=True,ax=ax,legend=False)


# plot x axis as an int
locator = matplotlib.ticker.MultipleLocator(2)

plt.gca().xaxis.set_major_locator(locator)
formatter = matplotlib.ticker.StrMethodFormatter("{x:.0f}")
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(20))
plt.xticks(rotation=90)

plt.suptitle(suptitle)
plt.tight_layout()

In [ ]:
ccd_visit_table1.plot.hist(column=["nightObs"], by="band", bins=200,figsize=(10, 8),grid=True,label=False)
plt.suptitle(suptitle)
plt.tight_layout()


In [ ]:
ccd_visit_table_counts1 = ccd_visit_table1.groupby(['nightObs','band']).count()["visitId"]
ccd_visit_table_counts1

#### Select one night and plot Zero point and Airmass

In [ ]:
nightObs_selected = 20230803
ccd_visit_table1_sel = ccd_visit_table1[ccd_visit_table1.nightObs == nightObs_selected]

In [ ]:
df = ccd_visit_table1_sel

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(16,10))

df[df["band"]=="g"].plot(x="Time",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax1,grid=True,legend=False)
ax1.set_title("zeroPoint")

df[df["band"]=="g"].plot(x="Time",y="airmass",marker='o',c="g",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="airmass",marker='o',c="r",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="airmass",marker='o',c="y",lw=0.0,ax=ax2,grid=True,legend=False)
ax2.set_title("airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))

df[df["band"]=="g"].plot.scatter(x="airmass",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="r"].plot(x="airmass",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="i"].plot(x="airmass",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax,grid=True,legend=False)
ax.set_title("zero point vs airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=10)
plt.tight_layout()

### Plot number of visit per day for patch 313

In [ ]:
tract_selected = 5615
patch_selected = 313
suptitle = f"CCD visit in tract {tract_selected} and patch {patch_selected}"

In [ ]:
ccd_visit_table2 = ccd_visit_table
ccd_visit_table2["selectedFlag"] = ccd_visit_table2[["tractID","patchID"]].apply(lambda x: x[0] == tract_selected and x[1] == patch_selected,axis =1, raw= True )


In [ ]:
ccd_visit_table_selected2 = ccd_visit_table1[ccd_visit_table2["selectedFlag"]]
ccd_visit_table_selected2.drop(["selectedFlag"],axis=1,inplace=True) 

In [ ]:
#ccd_visit_table2["dayobsMJD"]=ccd_visit_table2["obsStartMJD"].apply(np.int64)
#ccd_visit_table2['nightObs'] =ccd_visit_table2.apply(lambda x: x['visitId']//100_000, axis=1)

In [ ]:
import matplotlib.ticker
fig,ax = plt.subplots(1,1,figsize=(16,12))
ccd_visit_table2.plot.hist(column=["nightObs"], by="band", bins=200,grid=True,ax=ax,legend=False)


# plot x axis as an int
locator = matplotlib.ticker.MultipleLocator(2)

plt.gca().xaxis.set_major_locator(locator)
formatter = matplotlib.ticker.StrMethodFormatter("{x:.0f}")
plt.gca().xaxis.set_major_formatter(formatter)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(20))
plt.xticks(rotation=90)

plt.suptitle(suptitle)
plt.tight_layout()

## Select one night

In [ ]:
nightObs_selected = 20230803
ccd_visit_table1_sel = ccd_visit_table1[ccd_visit_table1.nightObs == nightObs_selected]
ccd_visit_table2_sel = ccd_visit_table2[ccd_visit_table2.nightObs == nightObs_selected]

In [ ]:
tract_selected = 5615
patch_selected = 294
#patch_selected = 313
suptitle = f"CCD visit in tract {tract_selected} and patch {patch_selected}"
df = ccd_visit_table1_sel

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(16,10))

df[df["band"]=="g"].plot(x="Time",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax1,grid=True,legend=False)
ax1.set_title("zeroPoint")

df[df["band"]=="g"].plot(x="Time",y="airmass",marker='o',c="g",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="airmass",marker='o',c="r",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="airmass",marker='o',c="y",lw=0.0,ax=ax2,grid=True,legend=False)
ax2.set_title("airmass")


#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))

df[df["band"]=="g"].plot.scatter(x="airmass",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="r"].plot(x="airmass",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="i"].plot(x="airmass",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax,grid=True,legend=False)
ax.set_title("zero point vs airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=10)
plt.tight_layout()

In [ ]:
tract_selected = 5615
patch_selected = 313
suptitle = f"CCD visit in tract {tract_selected} and patch {patch_selected}"
df = ccd_visit_table2_sel

In [ ]:
fig,(ax1,ax2) = plt.subplots(2,1,figsize=(16,10))

df[df["band"]=="g"].plot(x="Time",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax1,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax1,grid=True,legend=False)
ax1.set_title("zeroPoint")

df[df["band"]=="g"].plot(x="Time",y="airmass",marker='o',c="g",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="r"].plot(x="Time",y="airmass",marker='o',c="r",lw=0.0,ax=ax2,grid=True,legend=False)
df[df["band"]=="i"].plot(x="Time",y="airmass",marker='o',c="y",lw=0.0,ax=ax2,grid=True,legend=False)
ax2.set_title("airmass")


#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=16)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))

df[df["band"]=="g"].plot.scatter(x="airmass",y="zeroPoint",marker='o',c="g",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="r"].plot(x="airmass",y="zeroPoint",marker='o',c="r",lw=0.0,ax=ax,grid=True,legend=False)
df[df["band"]=="i"].plot(x="airmass",y="zeroPoint",marker='o',c="y",lw=0.0,ax=ax,grid=True,legend=False)
ax.set_title("zero point vs airmass")

#df.plot(x="Time",y="coord_dec",marker='o',c="g",lw=0.0,ax=ax2,grid=True)
plt.suptitle(suptitle + f" night {nightObs_selected}",y=1.0,fontsize=10)
plt.tight_layout()

# save CCD Visit

In [ ]:
ccd_visit_table

In [ ]:
ccd_visit_table.to_csv(filevisit_out)